In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets


import torch.optim as op
from torch.utils.tensorboard import SummaryWriter

In [4]:
import numpy as np
import scipy
import cv2
import random

In [5]:
import numpy.linalg as linalg
import matplotlib.pyplot as plt

In [84]:
#io
import glob
from PIL import Image
import os
import collections
import re
import sys
import datetime
import collections

In [8]:
# Add module path to sys
module_path = os.path.abspath(os.path.join('..'))
print("System path: "+module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
import micro_dl.torch_unet.networks.Unet2D as Unet2D
import micro_dl.torch_unet.networks.Unet25D as Unet25D
import micro_dl.torch_unet.networks.layers.ConvBlock2D as ConvBlock2D
import micro_dl.torch_unet.networks.layers.ConvBlock3D as ConvBlock3D
import micro_dl.torch_unet.utils.dataset as ds
import micro_dl.torch_unet.utils.io as io

System path: /home/christian.foley/virtual_staining/microDL


### Data Acquisition

In [27]:
data_directory_old = '/hpc/projects/compmicro/projects/virtualstaining/MBL_DL_image_translation/data/'

data_directory = '/hpc/projects/CompMicro/projects/Rickettsia/2022_RickettsiaAnalysis_Soorya/3_Cell_Image_Preprocessing/VirtualStainingMicroDL_KidneyMBL/data'
model_dir = f'/hpc/projects/compmicro/projects/virtualstaining/torch_microDL_models/Unet25d'

In [39]:
test_data_dir = '/hpc/projects/CompMicro/projects/virtualstaining/MBL_DL_image_translation/temp/tile_small_25D/tiles_256-256_step_128-128'

In [45]:
f = 'im_c002_z013_t000_p007_r1280-1536_c128-384_sl0-5.npy'

In [89]:
re.search('c[0-9]{3}_', f).group(0)

'c002_'

In [167]:
def sort_files(files, tags):
    '''
    Sorts filenames by priority order of tags specified in [tags]. 
    
    Params:
        files -> list[string]: list of filenames with uniform naming convention
        tags -> list[string]: tags to organize the filenames by    
    '''
    spans = []
    for tag in tags:
        span = re.search(tag, files[0]).span()
        spans.append(span)

    key = lambda x: tuple([x[s[0]:s[1]] for s in spans])
    files.sort(key = key)
    return files

def make_hash(files, tags):
    '''
    Recursively constructs hashmaps trees of filenames based on the tags given in tags.
    Follows order of tags given in tags; first tag -> separation in highest level tree 
    
    Params:
        files -> list[string]: list of filenames with uniform naming convention
        tags -> list[string]: tags to organize the filenames by
    '''
    
    if len(tags) == 0:
        return files, len(files)
    
    tag = tags.pop(0)
    hmap = {}
    
    for f in files:
        match = re.search(tag, f).group(0)
        if match in hmap:
            hmap[match].append(f)
        else:
            hmap[match] = [f]
    
    all_files = 0
    for key in hmap:
        key_files = hmap[key]
        hmap[key], n_files= make_hash(key_files, tags)
        all_files += n_files
    
    return hmap, all_files

def parse_directory(directory, mode = 'sort', tags = [], specify_type = None):
    '''
    Organizes filenames by the given tags in order. Tags should be given as regex patterns and will
    be recognized as the first instance of that pattern in each name.
    
    Any 4 character pattern can be specified in tags, but standard convention requires data of naming convention:
        'im_c***_z***_t***_p***_..._****.npy'
        where:
            c*** : channel
            z*** : z slice
            t*** : timestep
            p*** : position (fov)
    
    mode:
        - sort: returns all filenames of type specify_type sorted in order of tags
        - hash: returns nested hashmaps of files organized by their tags.
    
    Dependencies: re, os
    '''
    
    files = os.listdir(directory)
    
    # filter out incorrect types/endings
    if specify_type:
        if specify_type == 'dir':
            files = [f for f in files if os.path.isdir(os.path.join(directory,f))]
        elif specify_type == 'file':
            files = [f for f in files if os.path.isfile(os.path.join(directory,f))]
        else:
            files = [f for f in files if os.path.isfile(os.path.join(directory,f))]
            
            ind = len(specify_type)
            files = [f for f in files if f[-ind:] == specify_type]
        
        
    if mode == 'sort':
        sorted_files = sort_files(files, tags)
        return files
    
    if mode == 'hash':
        file_hash = make_hash(files, tags)
        return file_hash
    
    if mode == 'sorted hash':
        sorted_files = sort_files(files, tags)
        sorted_file_hash, _ = make_hash(sorted_files, tags)
        return sorted_file_hash, _
            
    raise NotImplementedError(f'Mode {mode} not yet implemented or not accepted')
    return

In [168]:
directory_tree, _ = parse_directory(test_data_dir, 'sorted hash', ['t[0-9]{3}', 'p[0-9]{3}', 'c[0-9]{3}', 'z[0-9]{3}'], '.npy')

In [169]:
print(_)

17370
